# 🔄 Workflow Concepts - Microsoft.Agents.AI.Workflows

This notebook explains the key concepts of building AI workflows using the Microsoft Agent Framework.

> 📝 **Hands-on Exercises**: After reviewing these concepts, complete the exercises in [EXERCISES.md](EXERCISES.md).

## Table of Contents

1. [What is a Workflow?](#1-what-is-a-workflow)
2. [Core Building Blocks](#2-core-building-blocks)
3. [Sequential Workflow](#3-sequential-workflow)
4. [Concurrent Workflow](#4-concurrent-workflow)
5. [Human-in-the-Loop Workflow](#5-human-in-the-loop-workflow)
6. [Workflow Events](#6-workflow-events)
7. [Best Practices](#7-best-practices)

## 1. What is a Workflow?

A **Workflow** in Microsoft.Agents.AI is a directed graph of **Executors** connected by **Edges**. Data flows through the workflow, being processed by each executor in sequence or in parallel.

```
┌───────────┐     ┌───────────┐     ┌───────────┐
│ Executor  │────▶│ Executor  │────▶│ Executor  │
│    A      │     │    B      │     │    C      │
└───────────┘     └───────────┘     └───────────┘
     │                                    │
     └─────────── Edge ───────────────────┘
```

### Key Benefits

| Benefit | Description |
|---------|-------------|
| **Composability** | Build complex workflows from simple, reusable components |
| **AI Integration** | Seamlessly integrate AI agents into the workflow |
| **Human Oversight** | Pause workflows to get human input or approval |
| **Streaming** | Process events as they occur, not just at completion |

## Setup: Install Required NuGet Packages

Run the following cell to install the required packages for Workflows and Azure OpenAI integration.

In [ ]:
// Install NuGet packages for Workflows (using .NET 9 compatible versions)
#r "nuget: Microsoft.Agents.AI.Workflows, 1.0.0-preview.25171.1"
#r "nuget: Microsoft.Agents.AI, 1.0.0-preview.25171.1"
#r "nuget: Microsoft.Extensions.AI.OpenAI, 9.5.0-preview.1.25265.7"
#r "nuget: Azure.AI.OpenAI, 2.3.0-beta.1"
#r "nuget: Azure.Identity, 1.17.1"
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.6"
#r "nuget: Microsoft.Extensions.Configuration.EnvironmentVariables, 9.0.6"

using System;
using System.IO;
using System.Text.Json;
using System.Threading;
using System.Threading.Tasks;
using System.Collections.Generic;
using Microsoft.Extensions.Configuration;
using Microsoft.Extensions.AI;
using Microsoft.Agents.AI;
using Microsoft.Agents.AI.Workflows;
using Azure.AI.OpenAI;
using Azure.Identity;

Console.WriteLine("✅ Packages loaded successfully!");

## Configuration Setup

Configure your Azure OpenAI credentials. You can either:
1. Set them directly in the cell below, or
2. Use the `appsettings.Local.json` file in the `dotnet` folder

In [ ]:
using System.IO;
using Microsoft.Extensions.Configuration;

// Helper function to find the dotnet folder by traversing up the directory tree
string FindConfigPath(string startPath)
{
    var currentDir = new DirectoryInfo(startPath);

    // Traverse up to find the 'dotnet' folder
    while (currentDir != null)
    {
        if (currentDir.Name.Equals("dotnet", StringComparison.OrdinalIgnoreCase))
        {
            return currentDir.FullName;
        }
        currentDir = currentDir.Parent;
    }

    // Fallback to start path if dotnet folder not found
    return startPath;
}

// Load configuration from appsettings.Local.json in the dotnet folder
var configPath = FindConfigPath(Directory.GetCurrentDirectory());
Console.WriteLine($"📁 Config path: {configPath}");

var configuration = new ConfigurationBuilder()
    .SetBasePath(configPath)
    .AddJsonFile("appsettings.Local.json", optional: true)
    .AddEnvironmentVariables()
    .Build();

// Get Azure OpenAI settings - support both flat and nested config keys
var endpoint = configuration["AZURE_OPENAI_ENDPOINT"] 
    ?? configuration["AzureOpenAI:Endpoint"]
    ?? configuration["AZURE_AI_PROJECT_ENDPOINT"];
    
var apiKey = configuration["AZURE_OPENAI_API_KEY"] 
    ?? configuration["AzureOpenAI:ApiKey"];
    
var deploymentName = configuration["AZURE_OPENAI_DEPLOYMENT_NAME"] 
    ?? configuration["AzureOpenAI:DeploymentName"]
    ?? configuration["AZURE_AI_MODEL_DEPLOYMENT_NAME"]
    ?? "gpt-4o-mini";

// Get Service Principal credentials
var tenantId = configuration["AZURE_TENANT_ID"] ?? configuration["AzureOpenAI:TenantId"];
var clientId = configuration["AZURE_CLIENT_ID"] ?? configuration["AzureOpenAI:ClientId"];
var clientSecret = configuration["AZURE_CLIENT_SECRET"] ?? configuration["AzureOpenAI:ClientSecret"];

if (string.IsNullOrEmpty(endpoint))
{
    Console.WriteLine("⚠️ Azure OpenAI endpoint not configured");
    Console.WriteLine("   Set AZURE_OPENAI_ENDPOINT in appsettings.Local.json or environment");
}
else
{
    Console.WriteLine($"✅ Configuration loaded");
    Console.WriteLine($"   Endpoint: {endpoint}");
    Console.WriteLine($"   Deployment: {deploymentName}");
    
    if (!string.IsNullOrEmpty(apiKey))
        Console.WriteLine($"   Auth: API Key");
    else if (!string.IsNullOrEmpty(tenantId) && !string.IsNullOrEmpty(clientId) && !string.IsNullOrEmpty(clientSecret))
        Console.WriteLine($"   Auth: Service Principal");
    else
        Console.WriteLine($"   Auth: Azure CLI / DefaultAzureCredential");
}

In [ ]:
AzureOpenAIClient aoaiClient;

if (string.IsNullOrEmpty(endpoint))
{
    throw new InvalidOperationException("Azure OpenAI endpoint is not configured. Set AZURE_OPENAI_ENDPOINT in appsettings.Local.json or environment.");
}

if (!string.IsNullOrWhiteSpace(apiKey))
{
    // API Key authentication
    aoaiClient = new AzureOpenAIClient(new Uri(endpoint), new System.ClientModel.ApiKeyCredential(apiKey));
    Console.WriteLine("✅ Created Azure OpenAI client with API Key authentication");
}
else if (!string.IsNullOrWhiteSpace(tenantId) && !string.IsNullOrWhiteSpace(clientId) && !string.IsNullOrWhiteSpace(clientSecret))
{
    // Service Principal authentication
    var credential = new ClientSecretCredential(tenantId, clientId, clientSecret);
    aoaiClient = new AzureOpenAIClient(new Uri(endpoint), credential);
    Console.WriteLine("✅ Created Azure OpenAI client with Service Principal authentication");
}
else
{
    // Fallback to DefaultAzureCredential
    aoaiClient = new AzureOpenAIClient(new Uri(endpoint), new DefaultAzureCredential());
    Console.WriteLine("✅ Created Azure OpenAI client with DefaultAzureCredential");
}

var chatClient = aoaiClient.GetChatClient(deploymentName).AsIChatClient();
Console.WriteLine($"✅ Chat client created for deployment: {deploymentName}");

## Define the SupportTicket Model

Define the SupportTicket record that will be used throughout the workflow examples.

In [ ]:
// Define the SupportTicket model
public enum TicketPriority { Low, Medium, High, Critical }

public record SupportTicket(
    string TicketId,
    string CustomerId,
    string CustomerName,
    string Subject,
    string Description,
    TicketPriority Priority = TicketPriority.Medium
);

// Create a sample ticket for testing
var sampleTicket = new SupportTicket(
    TicketId: "TKT-12345",
    CustomerId: "CUST-001",
    CustomerName: "John Smith",
    Subject: "Cannot login to my account",
    Description: "I've been trying to login for the past hour but keep getting an 'invalid credentials' error even though I just reset my password.",
    Priority: TicketPriority.High
);

Console.WriteLine($"✅ Sample ticket created: {sampleTicket.TicketId} - {sampleTicket.Subject}");

## 2. Core Building Blocks

### 2.1 Executors

An **Executor** is a unit of work in a workflow. It receives input, processes it, and optionally sends output to the next executor.

In [ ]:
// ===== Class-Based Executor (Single Input Type) =====
// Executor that receives a SupportTicket and processes it

internal sealed class TicketIntakeExecutor() : Executor<SupportTicket>("TicketIntake")
{
    public override async ValueTask HandleAsync(SupportTicket ticket, IWorkflowContext context)
    {
        // Validate the ticket
        if (string.IsNullOrWhiteSpace(ticket.Subject))
            throw new ArgumentException("Subject is required");

        // Format the ticket for the AI agent
        var ticketText = $"Ticket ID: {ticket.TicketId}\nSubject: {ticket.Subject}\nDescription: {ticket.Description}";

        // Send as a user message to the next executor (AI agent)
        await context.SendMessageAsync(new ChatMessage(ChatRole.User, ticketText));
        
        // Trigger the AI agent to process the message
        await context.SendMessageAsync(new TurnToken(emitEvents: true));
    }
}

// ===== Executor Processing AI Agent Output =====
// Executor that receives output from an AI agent (List<ChatMessage>)

internal sealed class CategorizationBridgeExecutor() : Executor<List<ChatMessage>>("CategorizationBridge")
{
    public override async ValueTask HandleAsync(List<ChatMessage> messages, IWorkflowContext context)
    {
        // Get the AI's response
        var categorizationResult = messages.LastOrDefault()?.Text ?? "Unknown";

        // Prepare for next AI agent
        var prompt = $"Based on this categorization: {categorizationResult}\nGenerate a helpful customer response.";
        
        await context.SendMessageAsync(new ChatMessage(ChatRole.User, prompt));
        await context.SendMessageAsync(new TurnToken(emitEvents: true));
    }
}

// ===== Executor That Yields Output =====
// Executor that produces the final workflow output

internal sealed class ResponseBridgeExecutor() : Executor<List<ChatMessage>>("ResponseBridge")
{
    public override async ValueTask HandleAsync(List<ChatMessage> messages, IWorkflowContext context)
    {
        var response = messages.LastOrDefault()?.Text ?? "Unable to generate response.";
        
        // Yield the final output
        await context.YieldOutputAsync(response);
    }
}

Console.WriteLine("✅ Executor classes defined: TicketIntakeExecutor, CategorizationBridgeExecutor, ResponseBridgeExecutor");

### 2.2 AI Agents (ChatClientAgent)

**ChatClientAgent** integrates AI models (like Azure OpenAI) into workflows.

| Concept | Description |
|---------|-------------|
| **ChatMessage** | User or assistant message sent to/from the AI |
| **TurnToken** | Triggers the AI agent to process accumulated messages |
| **emitEvents** | When true, the agent emits completion events |
| **List&lt;ChatMessage&gt;** | Output type from AI agents containing all messages |

In [ ]:
// Create an AI agent with specific instructions
ChatClientAgent categorizationAgent = new(
    chatClient,                    // IChatClient from Azure OpenAI
    name: "CategorizationAgent",
    instructions: """
        You are a customer support ticket categorization specialist.
        Analyze the incoming support ticket and categorize it into:
        - BILLING: Payment issues, invoices, subscription
        - TECHNICAL: Software bugs, errors, how-to questions
        - GENERAL: Account inquiries, feedback
        
        Respond with JSON: {"category": "CATEGORY", "priority": "HIGH|MEDIUM|LOW"}
        """
);

Console.WriteLine($"✅ Created AI Agent: {categorizationAgent.Name}");

### 2.3 Workflow Builder

The **WorkflowBuilder** connects executors with edges to create a workflow.

| Edge Type | Method | Description |
|-----------|--------|-------------|
| **Direct** | `AddEdge(a, b)` | Simple A → B connection |
| **Fan-Out** | `AddFanOutEdge(a, [b, c, d])` | A → [B, C, D] (parallel) |
| **Fan-In** | `AddFanInEdge([a, b, c], d)` | [A, B, C] → D (aggregate) |

### 2.4 Workflow Execution

Execute workflows and process events using streaming. Key event types:

| Event Type | Description | When Emitted |
|------------|-------------|--------------|
| `ExecutorCompletedEvent` | An executor finished processing | After each executor completes |
| `WorkflowOutputEvent` | Workflow produced final output | When `YieldOutputAsync` is called |
| `RequestInfoEvent` | Workflow needs external input | When a `RequestPort` is reached |

## 3. Sequential Workflow

A **Sequential Workflow** processes data through a linear pipeline where each step depends on the previous one.

```
┌─────────────┐    ┌──────────────────┐    ┌─────────────────────┐    ┌──────────────┐    ┌──────────────┐
│   Ticket    │───▶│  Categorization  │───▶│  Categorization     │───▶│   Response   │───▶│   Response   │
│   Intake    │    │   AI Agent       │    │  Bridge Executor    │    │   AI Agent   │    │    Bridge    │
└─────────────┘    └──────────────────┘    └─────────────────────┘    └──────────────┘    └──────────────┘
```

**Use Cases:** Document Processing, Order Processing, Content Pipeline, Support Tickets

In [ ]:
// ===== Sequential Workflow Implementation =====

// 1. Create executors (already defined above)
var ticketIntake = new TicketIntakeExecutor();
var categorizationBridge = new CategorizationBridgeExecutor();
var responseBridge = new ResponseBridgeExecutor();

// 2. Create AI agents
ChatClientAgent seqCategorizationAgent = new(
    chatClient,
    name: "CategorizationAgent",
    instructions: """
        You are a customer support ticket categorization specialist.
        Analyze the incoming support ticket and categorize it into:
        - BILLING, TECHNICAL, or GENERAL
        Respond with JSON: {"category": "CATEGORY", "priority": "HIGH|MEDIUM|LOW", "summary": "brief summary"}
        """
);

ChatClientAgent responseAgent = new(
    chatClient,
    name: "ResponseAgent",
    instructions: """
        You are a friendly customer support representative.
        Generate a helpful, empathetic response to the customer.
        Keep it concise (3-4 sentences).
        """
);

// 3. Build sequential workflow
var sequentialWorkflow = new WorkflowBuilder(ticketIntake)
    .AddEdge(ticketIntake, seqCategorizationAgent)          // Intake → AI Categorize
    .AddEdge(seqCategorizationAgent, categorizationBridge)  // AI → Bridge
    .AddEdge(categorizationBridge, responseAgent)           // Bridge → AI Respond
    .AddEdge(responseAgent, responseBridge)                 // AI → Output
    .WithOutputFrom(responseBridge)
    .Build();

Console.WriteLine("✅ Sequential workflow built!");
Console.WriteLine("   Flow: TicketIntake → CategorizationAgent → CategorizationBridge → ResponseAgent → ResponseBridge");

In [ ]:
// ===== Run Sequential Workflow =====

var testTicket = new SupportTicket(
    TicketId: "TICKET-001",
    CustomerId: "CUST-001",
    CustomerName: "John Doe",
    Subject: "Unable to process payment",
    Description: "I've been trying to complete my subscription renewal but the payment keeps failing. I've tried multiple cards.",
    Priority: TicketPriority.High
);

Console.WriteLine($"📩 Processing ticket: {testTicket.TicketId}");
Console.WriteLine($"   Subject: {testTicket.Subject}\n");

var seqRun = await InProcessExecution.StreamAsync(sequentialWorkflow, testTicket);

await foreach (WorkflowEvent evt in seqRun.WatchStreamAsync())
{
    switch (evt)
    {
        case ExecutorCompletedEvent completed:
            Console.WriteLine($"✓ [{completed.ExecutorId}] completed");
            if (completed.Data is string data && !string.IsNullOrWhiteSpace(data))
                Console.WriteLine($"   Output: {data.Substring(0, Math.Min(100, data.Length))}...");
            break;
            
        case WorkflowOutputEvent output:
            Console.WriteLine($"\n📤 Final Response:\n{output.Data}");
            break;
    }
}

## 4. Concurrent Workflow

A **Concurrent Workflow** distributes work to multiple executors simultaneously and aggregates results.

```
                         ┌──────────────────┐
                    ┌───▶│  Billing Expert  │────┐
┌─────────────┐     │    └──────────────────┘    │    ┌─────────────────┐
│   Start     │─────│                            │───▶│   Aggregation   │
│  Executor   │     │    ┌──────────────────┐    │    │    Executor     │
└─────────────┘     └───▶│ Technical Expert │────┘    └─────────────────┘
        │                                                     │
        └──────── Fan-Out ──────── Fan-In ────────────────────┘
```

**Use Cases:** Multi-Expert Analysis, Parallel Processing, Consensus Building, Redundancy

In [ ]:
// ===== Concurrent Workflow Executors =====

// Start Executor - broadcasts message to all connected agents
internal sealed class ConcurrentStartExecutor() : Executor<string>("ConcurrentStart")
{
    public override async ValueTask HandleAsync(string message, IWorkflowContext context)
    {
        // Broadcast the message to all connected agents
        await context.SendMessageAsync(new ChatMessage(ChatRole.User, message));

        // Broadcast the turn token to kick off all agents
        await context.SendMessageAsync(new TurnToken(emitEvents: true));
    }
}

// Aggregation Executor - collects results from multiple agents
internal sealed class ConcurrentAggregationExecutor() : Executor<List<ChatMessage>>("ConcurrentAggregation")
{
    private readonly List<ChatMessage> _messages = [];

    public override async ValueTask HandleAsync(List<ChatMessage> messages, IWorkflowContext context)
    {
        _messages.AddRange(messages);

        // Wait for responses from all agents (2 in this demo)
        if (_messages.Count >= 2)
        {
            var formattedMessages = string.Join(
                Environment.NewLine + Environment.NewLine,
                _messages.Select(m => $"[{m.AuthorName}]: {m.Text}")
            );

            await context.YieldOutputAsync(formattedMessages);
        }
    }
}

Console.WriteLine("✅ Concurrent workflow executors defined: ConcurrentStartExecutor, ConcurrentAggregationExecutor");

In [ ]:
// ===== Build Concurrent Workflow =====

// Create multiple AI agents (experts)
ChatClientAgent billingExpert = new(
    chatClient,
    name: "BillingExpert",
    instructions: """
        You are an expert in billing and subscription matters.
        Analyze the customer's question from a billing perspective.
        If not billing-related, briefly acknowledge and defer.
        Keep responses concise (2-3 sentences).
        """
);

ChatClientAgent technicalExpert = new(
    chatClient,
    name: "TechnicalExpert",
    instructions: """
        You are an expert in technical support and troubleshooting.
        Analyze the customer's question from a technical perspective.
        If not technical, briefly acknowledge and defer.
        Keep responses concise (2-3 sentences).
        """
);

// Create start and aggregation executors
var startExecutor = new ConcurrentStartExecutor();
var aggregationExecutor = new ConcurrentAggregationExecutor();

// Build concurrent workflow with fan-out/fan-in
var concurrentWorkflow = new WorkflowBuilder(startExecutor)
    .AddEdge(startExecutor, billingExpert)
    .AddEdge(startExecutor, technicalExpert)
    .AddEdge(billingExpert, aggregationExecutor)
    .AddEdge(technicalExpert, aggregationExecutor)
    .WithOutputFrom(aggregationExecutor)
    .Build();

Console.WriteLine("✅ Concurrent workflow built!");
Console.WriteLine("   Flow: Start → [BillingExpert, TechnicalExpert] → Aggregation");

In [ ]:
// ===== Run Concurrent Workflow =====

var customerQuestion = "My software keeps crashing and I was also charged twice last month. Can you help?";

Console.WriteLine($"📩 Customer Question: {customerQuestion}\n");

var concRun = await InProcessExecution.StreamAsync(concurrentWorkflow, customerQuestion);

await foreach (WorkflowEvent evt in concRun.WatchStreamAsync())
{
    switch (evt)
    {
        case ExecutorCompletedEvent completed:
            Console.WriteLine($"✓ [{completed.ExecutorId}] completed");
            break;
            
        case WorkflowOutputEvent output:
            Console.WriteLine($"\n📤 Aggregated Expert Responses:\n{output.Data}");
            break;
    }
}

## 5. Human-in-the-Loop Workflow

A **Human-in-the-Loop Workflow** pauses execution to get human input, approval, or oversight.

```
┌─────────────┐    ┌──────────────┐    ┌──────────────────┐    ┌──────────────┐
│   Ticket    │───▶│   AI Draft   │───▶│   RequestPort    │───▶│   Finalize   │
│   Intake    │    │    Agent     │    │ (Human Review)   │    │   Response   │
└─────────────┘    └──────────────┘    └──────────────────┘    └──────────────┘
                                               │
                                    ┌──────────┴───────────┐
                                    │     Supervisor       │
                                    │   [Approve/Edit/     │
                                    │    Escalate]         │
                                    └──────────────────────┘
```

**Use Cases:** Content Approval, Financial Decisions, Customer Escalation, Quality Control, Compliance

In [ ]:
// ===== Human-in-the-Loop Types and Executors =====
#nullable enable

// Define request and response types
public sealed record SupervisorReviewRequest(
    string TicketId,
    string Category,
    string Priority,
    string DraftResponse
);

public sealed record SupervisorDecision(
    ReviewAction Action,      // Approve, Edit, Escalate
    string? ModifiedResponse,
    string? Notes
);

public enum ReviewAction { Approve, Edit, Escalate }

// Ticket intake for HITL workflow
internal sealed class HitlTicketIntakeExecutor() : Executor<SupportTicket>("HitlTicketIntake")
{
    public static SupportTicket? CurrentTicket { get; private set; }
    
    public override async ValueTask HandleAsync(SupportTicket ticket, IWorkflowContext context)
    {
        CurrentTicket = ticket;
        var ticketText = $"Ticket ID: {ticket.TicketId}\nCustomer: {ticket.CustomerName}\nSubject: {ticket.Subject}\nDescription: {ticket.Description}";
        await context.SendMessageAsync(new ChatMessage(ChatRole.User, ticketText));
        await context.SendMessageAsync(new TurnToken(emitEvents: true));
    }
}

// Bridge that creates review request
internal sealed class DraftBridgeExecutor() : Executor<List<ChatMessage>>("DraftBridge")
{
    public override async ValueTask HandleAsync(List<ChatMessage> messages, IWorkflowContext context)
    {
        var draftResponse = messages.LastOrDefault()?.Text ?? "Unable to generate.";
        var ticket = HitlTicketIntakeExecutor.CurrentTicket!;

        var reviewRequest = new SupervisorReviewRequest(
            TicketId: ticket.TicketId,
            Category: "BILLING",
            Priority: ticket.Priority.ToString(),
            DraftResponse: draftResponse
        );

        // Send to RequestPort - workflow will pause here
        await context.SendMessageAsync(reviewRequest);
    }
}

// Finalize executor that uses supervisor decision
internal sealed class FinalizeExecutor() : Executor<SupervisorDecision>("Finalize")
{
    public override async ValueTask HandleAsync(SupervisorDecision decision, IWorkflowContext context)
    {
        var finalResponse = decision.Action switch
        {
            ReviewAction.Approve => $"[APPROVED] {decision.Notes}",
            ReviewAction.Edit => $"[EDITED] {decision.ModifiedResponse}",
            ReviewAction.Escalate => $"[ESCALATED] {decision.Notes}",
            _ => "[UNKNOWN ACTION]"
        };
        
        await context.YieldOutputAsync(finalResponse);
    }
}

Console.WriteLine("✅ Human-in-the-Loop types and executors defined");

In [ ]:
// ===== Build Human-in-the-Loop Workflow =====
// NOTE: The RequestPort API requires .NET 10 preview packages (Microsoft.Agents.AI.Workflows 1.0.0-preview.260108.1)
// This section is disabled when using .NET 9 compatible packages.

Console.WriteLine("⚠️ Human-in-the-Loop workflow requires .NET 10 preview packages");
Console.WriteLine("   The RequestPort API is not available in .NET 9 compatible versions.");
Console.WriteLine("   To enable this feature, update packages to:");
Console.WriteLine("   - Microsoft.Agents.AI.Workflows: 1.0.0-preview.260108.1");
Console.WriteLine("   - Microsoft.Extensions.AI.OpenAI: 10.1.1-preview.1.25612.2");

/*
// Create AI agent for drafting
ChatClientAgent draftAgent = new(
    chatClient,
    name: "DraftAgent",
    instructions: "You are an experienced customer support specialist..."
);

// Create executors
var hitlTicketIntake = new HitlTicketIntakeExecutor();
var draftBridge = new DraftBridgeExecutor();
var supervisorReviewPort = Microsoft.Agents.AI.Workflows.RequestPort.Create<SupervisorReviewRequest, SupervisorDecision>("SupervisorReview");
var finalizeExecutor = new FinalizeExecutor();

// Build workflow
var hitlWorkflow = new WorkflowBuilder(hitlTicketIntake)
    .AddEdge(hitlTicketIntake, draftAgent)
    .AddEdge(draftAgent, draftBridge)
    .AddEdge(draftBridge, supervisorReviewPort)
    .AddEdge(supervisorReviewPort, finalizeExecutor)
    .WithOutputFrom(finalizeExecutor)
    .Build();
*/

In [ ]:
// ===== Run Human-in-the-Loop Workflow =====
// NOTE: Disabled - requires .NET 10 preview packages

Console.WriteLine("⚠️ Human-in-the-Loop workflow execution skipped (requires .NET 10 preview packages)");

/*
var refundTicket = new SupportTicket(
    TicketId: "TICKET-002",
    CustomerId: "CUST-002",
    CustomerName: "Jane Smith",
    Subject: "Request for refund",
    Description: "I purchased the premium plan 5 days ago but it doesn't have the features I expected. I'd like a full refund please.",
    Priority: TicketPriority.High
);

Console.WriteLine($"📩 Processing ticket: {refundTicket.TicketId}");
Console.WriteLine($"   Subject: {refundTicket.Subject}\n");

var hitlRun = await InProcessExecution.StreamAsync(hitlWorkflow, refundTicket);

await foreach (WorkflowEvent evt in hitlRun.WatchStreamAsync())
{
    switch (evt)
    {
        case ExecutorCompletedEvent completed:
            Console.WriteLine($"✓ [{completed.ExecutorId}] completed");
            break;
            
        case RequestInfoEvent requestEvt:
            var request = requestEvt.Request.DataAs<SupervisorReviewRequest>();
            
            Console.WriteLine($"\n⏸️ HUMAN REVIEW REQUIRED");
            Console.WriteLine($"   Ticket: {request.TicketId}");
            Console.WriteLine($"   Category: {request.Category}");
            Console.WriteLine($"   Priority: {request.Priority}");
            Console.WriteLine($"   Draft Response:\n   {request.DraftResponse}\n");
            
            Console.WriteLine("   [Simulating supervisor approval...]");
            var decision = new SupervisorDecision(
                Action: ReviewAction.Approve,
                ModifiedResponse: null,
                Notes: "Response looks good. Approved for sending."
            );
            
            var response = requestEvt.Request.CreateResponse(decision);
            await hitlRun.SendResponseAsync(response);
            Console.WriteLine("   ✓ Supervisor decision sent, workflow resuming...\n");
            break;
            
        case WorkflowOutputEvent output:
            Console.WriteLine($"\n📤 Final Output: {output.Data}");
            break;
    }
}
*/

## 6. Workflow Events Reference

| Event Type | Description | When Emitted |
|------------|-------------|---------------|
| `ExecutorCompletedEvent` | An executor finished processing | After each executor completes |
| `WorkflowOutputEvent` | Workflow produced final output | When `YieldOutputAsync` is called |
| `RequestInfoEvent` | Workflow needs external input | When a `RequestPort` is reached |

## 7. Best Practices

### Executor Design
- **Single Responsibility**: Each executor should do one thing well
- **Stateless When Possible**: Avoid storing state in executors
- **Clear Naming**: Use descriptive names that indicate purpose
- **Bridge Executors**: Use bridge executors between AI agents to transform data

### AI Agent Instructions
- Be specific about expected output format (JSON recommended)
- Keep instructions concise and focused
- Specify edge cases and fallback behavior

### Message Flow Pattern
```csharp
// Standard pattern for sending to AI agents:
await context.SendMessageAsync(new ChatMessage(ChatRole.User, prompt), ct);
await context.SendMessageAsync(new TurnToken(emitEvents: true), ct);

// Standard pattern for yielding output:
await context.YieldOutputAsync(result, ct);
```

## Summary

| Pattern | When to Use | Key Methods |
|---------|-------------|-------------|
| **Sequential** | Linear processing pipeline | `AddEdge()` |
| **Concurrent** | Parallel processing, multi-expert | `AddFanOutEdge()`, `AddFanInEdge()` |
| **Human-in-the-Loop** | Approval, oversight, exceptions | `RequestPort.Create<>()` |

### Quick Reference
```csharp
// Create AI agent
ChatClientAgent agent = new(chatClient, name: "Agent", instructions: "...");

// Build workflow
var workflow = new WorkflowBuilder(entryExecutor)
    .AddEdge(a, b)                          // Sequential
    .AddFanOutEdge(a, [b, c])               // Parallel (fan-out)
    .AddFanInEdge([b, c], d)                // Aggregate (fan-in)
    .WithOutputFrom(outputExecutor)
    .Build();

// Execute with streaming
await using var run = await InProcessExecution.StreamAsync(workflow, input);
await foreach (var evt in run.WatchStreamAsync()) { /* handle events */ }
```